In [1]:
#Functional API fusion model
import keras
import tensorflow
tensorflow.debugging.set_log_device_placement(False)
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.models import Model
#from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import BatchNormalization
from keras.layers import Conv1D
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import Conv3D

from keras.layers import ConvLSTM2D
from keras.layers import ConvLSTM1D
from keras.layers import LSTM
from keras.layers import GRU

from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.layers import MaxPooling1D
from keras.layers import MaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import Concatenate
from keras.layers import Input

from tensorflow.keras import optimizers
from keras.utils.np_utils import to_categorical
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix
import datetime
from sklearn.model_selection import KFold
import pickle
from sklearn.utils import shuffle
import sys


In [2]:

#import tensorflow as tf; print(tf.__version__)
#import keras

#from keras.layers import ConvLSTM3D

In [3]:
gpus = tensorflow.config.list_physical_devices('GPU')


strategy = tensorflow.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1","/gpu:2", "/gpu:3"])#devices=["/gpu:0", "/gpu:1"]

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')


In [4]:
load = np.load('data/RAVDESS_basic.npz',allow_pickle=True)
audio = load['audio']
video = load['video']
labels = load['labels']
n_splits = 6
print(np.array(video.shape))
print(np.array(audio.shape))
print(np.array(labels.shape))

[1152  115  128  128]
[  1152 110544]
[1152]


In [5]:
#video pre-process
fraction = 5
frames = int(video.shape[1]/fraction)
shape = (128,128,frames,1)


reshape_videos = []

for i in video:
    single_video = []
    for j in range( i.shape[0] ): #HEREi.shape[0]
        if(  j % fraction == 0 ):
            single_video.append( i[j] )
    reshape_videos.append( single_video )

video = np.array(reshape_videos)
del reshape_videos


video_input = (frames, 128,  128, 1)
video_shape = (video.shape[0], frames, 128, 128, 1)
video = np.reshape(video, video_shape)
print(video.shape)

(1152, 23, 128, 128, 1)


In [6]:
#audio pre-process 1dconv
num_rows = audio.shape[1]
num_columns = 1
num_channels = 1

audio_input = (num_rows, num_columns)
audio_shape = (audio.shape[0], num_rows, num_columns)
audio = np.reshape(audio, audio_shape)

print(audio.shape)

(1152, 110544, 1)


In [7]:
#audio pre-process mfcc
num_rows = audio.shape[1]
num_columns = audio.shape[2]
num_channels = 1
audio_input = (num_rows, num_columns)
audio_shape = (audio.shape[0], num_rows, num_columns)
audio = np.reshape(audio, audio_shape)

print(audio.shape)

(1152, 110544, 1)


In [8]:
#labels = labels - 1
labels = to_categorical(labels)
target_names = ['neutral', 'calm', 'happy', 'sad', 'angry', 'fearful', 'disgust', 'surprised']


In [19]:
#k-fold testing video
n = 0 #current fold

epochs = 30
name = 'ravdess_' + str(epochs) + '_video'
pickle_file = 'pickle_files/' + name + '.pckl'

training_score = []
testing_score = []

f = open(pickle_file, 'wb') #file for saving information/data about each fold


kf = KFold(n_splits,shuffle=True,random_state=42)
kf.get_n_splits(audio)


kf = KFold(n_splits,shuffle=True,random_state=42)
kf.get_n_splits(audio)

for train_index, test_index in kf.split(audio):
    print('Beginning fold #' + str(n) + '...')
    #print(train_index)
    #print(test_index)
    a_train, a_test = audio[train_index], audio[test_index]
    v_train, v_test = video[train_index], video[test_index]
    l_train, l_test = labels[train_index], labels[test_index]


    with strategy.scope():
        
        model = Sequential()
        
        model.add(Conv3D(64, kernel_size=(3,3,3), strides=(2,2,1),input_shape=video_input))
        model.add(LeakyReLU(alpha=0.2))

        #layer 2
        model.add(Conv3D(128, kernel_size=(3,3,3), strides=(2,2,1), padding='SAME'))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))

        #layer 3
        model.add(Conv3D(256, kernel_size=(3,3,3), strides=(2,2,1), padding='SAME'))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))

        #layer 4
        model.add(Conv3D(512, kernel_size=(3,3,3), strides=(2,2,2), padding='SAME'))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))

        #layer 5
        model.add(Conv3D(1024, kernel_size=(3,3,3), strides=(2,2,2), padding='SAME'))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))
        
        #layer 6
        #model.add(Conv3D(2048, kernel_size=(3,3,3), strides=(2,2,2), padding='SAME'))
        #model.add(BatchNormalization())
        #model.add(LeakyReLU(alpha=0.2))
        
       
        #model.add(ConvLSTM2D(256,kernel_size=(2,2), strides=(1,1),padding='SAME', dropout=0.2))
       
        #layer 7
        
        model.add(Flatten())
        
        
        model.add(Dense(6, activation='softmax'))
        
                
        model.compile(loss='categorical_crossentropy', #categorical_crossentropy
                        optimizer=optimizers.Adam(learning_rate=0.00001), 
                                 metrics=['accuracy'])
        model.fit(v_train, l_train,
                        batch_size=30, #batch 9
                        epochs=epochs,
                        verbose=1,
                        validation_split=0.166)
        
        score = model.evaluate(v_train, l_train, verbose=0)
        pickle.dump([score, 'training accuracy fold #' + str(n)], f)
        training_score.append(score)

        score = model.evaluate(v_test, l_test, verbose=0)
        pickle.dump([score, 'testing accuracy fold #' + str(n)], f)
        testing_score.append(score)

        Y_pred = model.predict(v_test)
        y_pred = np.argmax(Y_pred, axis=1)
        labels_ = np.argmax(l_test, axis=1)
        pickle.dump(confusion_matrix(labels_, y_pred), f)
        pickle.dump(classification_report(labels_, y_pred), f)
        
        n+=1
        
np.savez('data/'+name,x=training_score,y=testing_score)
f.close()

Beginning fold #0...
Epoch 1/30
27/27 [==============================] - 18s 364ms/step - loss: 1.9631 - accuracy: 0.2338 - val_loss: 1.8718 - val_accuracy: 0.1688
Epoch 2/30
27/27 [==============================] - 8s 286ms/step - loss: 0.9236 - accuracy: 0.7287 - val_loss: 1.8618 - val_accuracy: 0.1688
Epoch 3/30
27/27 [==============================] - 8s 287ms/step - loss: 0.5309 - accuracy: 0.9112 - val_loss: 1.8184 - val_accuracy: 0.1500
Epoch 4/30
27/27 [==============================] - 8s 290ms/step - loss: 0.2860 - accuracy: 0.9837 - val_loss: 1.8090 - val_accuracy: 0.1562
Epoch 5/30
27/27 [==============================] - 8s 290ms/step - loss: 0.1741 - accuracy: 0.9975 - val_loss: 1.7865 - val_accuracy: 0.1625
Epoch 6/30
27/27 [==============================] - 8s 289ms/step - loss: 0.1153 - accuracy: 1.0000 - val_loss: 1.7818 - val_accuracy: 0.2000
Epoch 7/30
27/27 [==============================] - 8s 287ms/step - loss: 0.0819 - accuracy: 1.0000 - val_loss: 1.7663 - val_a

27/27 [==============================] - 8s 288ms/step - loss: 0.0062 - accuracy: 1.0000 - val_loss: 2.5888 - val_accuracy: 0.2125
Epoch 29/30
27/27 [==============================] - 8s 288ms/step - loss: 0.0058 - accuracy: 1.0000 - val_loss: 2.6030 - val_accuracy: 0.2000
Epoch 30/30
27/27 [==============================] - 8s 286ms/step - loss: 0.0055 - accuracy: 1.0000 - val_loss: 2.5901 - val_accuracy: 0.2000
Beginning fold #2...
Epoch 1/30
27/27 [==============================] - 18s 362ms/step - loss: 1.9638 - accuracy: 0.2875 - val_loss: 1.8731 - val_accuracy: 0.1500
Epoch 2/30
27/27 [==============================] - 8s 287ms/step - loss: 0.9026 - accuracy: 0.7563 - val_loss: 1.8602 - val_accuracy: 0.1625
Epoch 3/30
27/27 [==============================] - 8s 288ms/step - loss: 0.4841 - accuracy: 0.9450 - val_loss: 1.8551 - val_accuracy: 0.1813
Epoch 4/30
27/27 [==============================] - 8s 287ms/step - loss: 0.2886 - accuracy: 0.9812 - val_loss: 1.8454 - val_accuracy: 

27/27 [==============================] - 8s 291ms/step - loss: 0.0072 - accuracy: 1.0000 - val_loss: 2.5668 - val_accuracy: 0.1562
Epoch 26/30
27/27 [==============================] - 8s 288ms/step - loss: 0.0067 - accuracy: 1.0000 - val_loss: 2.5680 - val_accuracy: 0.1562
Epoch 27/30
27/27 [==============================] - 8s 290ms/step - loss: 0.0063 - accuracy: 1.0000 - val_loss: 2.5818 - val_accuracy: 0.1562
Epoch 28/30
27/27 [==============================] - 8s 288ms/step - loss: 0.0059 - accuracy: 1.0000 - val_loss: 2.5994 - val_accuracy: 0.1562
Epoch 29/30
27/27 [==============================] - 8s 288ms/step - loss: 0.0055 - accuracy: 1.0000 - val_loss: 2.6135 - val_accuracy: 0.1562
Epoch 30/30
27/27 [==============================] - 8s 288ms/step - loss: 0.0053 - accuracy: 1.0000 - val_loss: 2.6139 - val_accuracy: 0.1562
Beginning fold #4...
Epoch 1/30
27/27 [==============================] - 18s 389ms/step - loss: 1.8429 - accuracy: 0.3137 - val_loss: 1.8580 - val_accurac

27/27 [==============================] - 8s 291ms/step - loss: 0.0099 - accuracy: 1.0000 - val_loss: 2.3603 - val_accuracy: 0.2688
Epoch 23/30
27/27 [==============================] - 8s 292ms/step - loss: 0.0091 - accuracy: 1.0000 - val_loss: 2.4013 - val_accuracy: 0.2625
Epoch 24/30
27/27 [==============================] - 8s 293ms/step - loss: 0.0084 - accuracy: 1.0000 - val_loss: 2.4240 - val_accuracy: 0.2688
Epoch 25/30
27/27 [==============================] - 8s 293ms/step - loss: 0.0079 - accuracy: 1.0000 - val_loss: 2.4273 - val_accuracy: 0.2688
Epoch 26/30
27/27 [==============================] - 8s 289ms/step - loss: 0.0073 - accuracy: 1.0000 - val_loss: 2.4371 - val_accuracy: 0.2562
Epoch 27/30
27/27 [==============================] - 8s 288ms/step - loss: 0.0068 - accuracy: 1.0000 - val_loss: 2.4644 - val_accuracy: 0.2688
Epoch 28/30
27/27 [==============================] - 8s 303ms/step - loss: 0.0065 - accuracy: 1.0000 - val_loss: 2.4932 - val_accuracy: 0.2625
Epoch 29/30

In [19]:
#k-fold testing audio
n = 0 #current fold

epochs = 30
name = 'ravdess_' + str(epochs) + '_lstm_audio'
pickle_file = 'pickle_files/' + name + '.pckl'

training_score = []
testing_score = []

f = open(pickle_file, 'wb') #file for saving information/data about each fold


kf = KFold(n_splits,shuffle=True,random_state=42)
kf.get_n_splits(audio)


kf = KFold(n_splits,shuffle=True,random_state=42)
kf.get_n_splits(audio)

for train_index, test_index in kf.split(audio):
    print('Beginning fold #' + str(n) + '...')
    #print(train_index)
    #print(test_index)
    a_train, a_test = audio[train_index], audio[test_index]
    v_train, v_test = video[train_index], video[test_index]
    l_train, l_test = labels[train_index], labels[test_index]


    with strategy.scope():
        model = Sequential()
        model.add(Conv1D(padding='same', filters=25, kernel_size=64, strides = 8, input_shape=audio_input))
        model.add(LeakyReLU(alpha=0.2))

        model.add(Conv1D(padding='same', filters=25, kernel_size=128, strides = 8))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))

        model.add(Conv1D(padding='same', filters=25, kernel_size=256, strides = 8))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))

        model.add(Conv1D(padding='same', filters=25, kernel_size=512, strides = 4))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))

        model.add(Conv1D(padding='same', filters=25, kernel_size=1024, strides = 4))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))

        model.add(GRU(128))
        
        model.add(Flatten())

        model.add(Dense(6, activation='softmax'))

        model.compile(loss='categorical_crossentropy', #categorical_crossentropy
                        optimizer=optimizers.Adam(learning_rate=0.00001), 
                                 metrics=['accuracy'])
        model.fit(a_train, l_train,
                        batch_size=30, #batch 9
                        epochs=epochs,
                        verbose=1,
                        validation_split=0.166)
        
        score = model.evaluate(a_train, l_train, verbose=0)
        pickle.dump([score, 'training accuracy fold #' + str(n)], f)
        training_score.append(score)

        score = model.evaluate(a_test, l_test, verbose=0)
        pickle.dump([score, 'testing accuracy fold #' + str(n)], f)
        testing_score.append(score)

        Y_pred = model.predict(a_test)
        y_pred = np.argmax(Y_pred, axis=1)
        labels_ = np.argmax(l_test, axis=1)
        pickle.dump(confusion_matrix(labels_, y_pred), f)
        pickle.dump(classification_report(labels_, y_pred), f)
        
        n+=1
    break
        
np.savez('data/'+name,x=training_score,y=testing_score)
f.close()

Beginning fold #0...
Epoch 1/30
INFO:tensorflow:batch_all_reduce: 23 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 23 all-reduces with algorithm = nccl, num_packs = 1
27/27 [==============================] - 19s 176ms/step - loss: 1.8037 - accuracy: 0.1750 - val_loss: 1.7918 - val_accuracy: 0.1688
Epoch 2/30
27/27 [==============================] - 1s 51ms/step - loss: 1.7619 - accuracy: 0.1975 - val_loss: 1.7918 - val_accuracy: 0.1562
Epoch 3/30
27/27 [==============================] - 1s 56ms/step - loss: 1.7334 - accuracy: 0.2512 - val_loss: 1.7919 - val_accuracy: 0.1562
Epoch 4/30
27/27 [==============================] - 2s 63ms/step - loss: 1.7226 - accuracy: 0.3000 - val_loss: 1.7920 - val_accuracy: 0.1562
Epoch 5/30
27/27 [==============================] - 1s 53ms/step - loss: 1.6946 - accuracy: 0.2937 - val_loss: 1.7921 - val_accuracy: 0.1562
Epoch 6/30
27/27 [==============================] - 1s 50ms/step - loss: 1.6930 - accuracy: 0.3113 -

KeyboardInterrupt: 

In [ ]:
#k-fold testing video
n = 0 #current fold

epochs = 30
name = 'ravdess_' + str(epochs) + '_lstm_video'
pickle_file = 'pickle_files/' + name + '.pckl'

training_score = []
testing_score = []

f = open(pickle_file, 'wb') #file for saving information/data about each fold


kf = KFold(n_splits,shuffle=True,random_state=42)
kf.get_n_splits(audio)

print(kf)

print(audio.shape)
print(labels.shape)

kf = KFold(n_splits,shuffle=True,random_state=42)
kf.get_n_splits(audio)

for train_index, test_index in kf.split(audio):
    print('Beginning fold #' + str(n) + '...')
    #print(train_index)
    #print(test_index)
    a_train, a_test = audio[train_index], audio[test_index]
    v_train, v_test = video[train_index], video[test_index]
    l_train, l_test = labels[train_index], labels[test_index]

    with strategy.scope(): 

        model = Sequential()
       
        model.add(ConvLSTM2D(512, kernel_size=(3,3), strides=(2,2), input_shape=video_input, data_format='channels_last',return_sequences=True))
        #model.add(LeakyReLU(alpha=0.2))

        model.add(ConvLSTM2D(256, kernel_size=(3,3), strides=(2,2), data_format='channels_last',return_sequences=True))# padding='SAME'
        #model.add(BatchNormalization())
        #model.add(LeakyReLU(alpha=0.2))

        #model.add(ConvLSTM2D(128, kernel_size=(3,3), strides=(2,2), data_format='channels_last',return_sequences=True))# padding='SAME'
        #model.add(BatchNormalization())
        #model.add(LeakyReLU(alpha=0.2))

        #model.add(ConvLSTM2D(64, kernel_size=(3,3), strides=(2,2), data_format='channels_last',return_sequences=True))# padding='SAME'
        #model.add(BatchNormalization())
        #model.add(LeakyReLU(alpha=0.2))

        #1024 runs out of memory
        #model.add(ConvLSTM2D(1024, kernel_size=(3,3), strides=(2,2), data_format='channels_last',return_sequences=True))# padding='SAME'
        #model.add(BatchNormalization())
        #model.add(LeakyReLU(alpha=0.2))

        #model.add(ConvLSTM2D(2048, kernel_size=(3,3), strides=(2,2), data_format='channels_last',return_sequences=True))# padding='SAME'
        #model.add(BatchNormalization())
        #model.add(LeakyReLU(alpha=0.2))


        model.add(Flatten())

        model.add(Dense(6, activation='softmax'))
        model.compile(loss='categorical_crossentropy', #categorical_crossentropy
                        optimizer=optimizers.Adam(learning_rate=0.0001), 
                                 metrics=['accuracy'])
        model.fit(v_train, l_train,
                        batch_size=30, #batch 9
                        epochs=epochs,
                        verbose=1,
                        validation_split=0.166)
        
        score = model.evaluate(v_train, l_train, verbose=0)
        pickle.dump([score, 'training accuracy fold #' + str(n)], f)
        training_score.append(score)

        score = model.evaluate(v_test, l_test, verbose=0)
        pickle.dump([score, 'testing accuracy fold #' + str(n)], f)
        testing_score.append(score)

        Y_pred = model.predict(v_test)
        y_pred = np.argmax(Y_pred, axis=1)
        labels_ = np.argmax(l_test, axis=1)
        pickle.dump(confusion_matrix(labels_, y_pred), f)
        pickle.dump(classification_report(labels_, y_pred), f)
        
        n+=1
        
np.savez('data/'+name,x=training_score,y=testing_score)
f.close()

In [20]:
#k_fold testing results
print('data/'+name+'.npz')
load = np.load('data/'+name+'.npz',allow_pickle=True)

training_score = load['x']
testing_score = load['y']



training_score = np.transpose(training_score)
testing_score = np.transpose(testing_score)


print('testing_score loss:', np.mean(testing_score[0]))
print('testing_score accuracy:',np.mean(testing_score[1]))



data/ravdess_30_video.npz
testing_score loss: 0.702016681432724
testing_score accuracy: 0.7673611144224802


In [ ]:
#k_fold testing results
print('data/'+name+'.npz')
load = np.load('data/'+'ravdess_30_lstm'+'.npz',allow_pickle=True)

training_score = load['x']
testing_score = load['y']



training_score = np.transpose(training_score)
testing_score = np.transpose(testing_score)


print('testing_score loss:', np.mean(testing_score[0]))
print('testing_score accuracy:',np.mean(testing_score[1]))

with (open('pickle_files/ravdess_30_lstm.pckl', "rb")) as openfile:
    while True:
        try:
            print(pickle.load(openfile))
        except EOFError:
            break


In [ ]:
final cnnlstm 20 filters
testing_score loss: 0.6053348133961359
testing_score accuracy: 0.7708333333333334

In [ ]:
first layer cnnlstm
testing_score loss: 0.6772788763046265
testing_score accuracy: 0.7612847288449606

ravdess_30_lstm full lstm architecture
testing_score loss: 0.7224234342575073
testing_score accuracy: 0.7569444477558136

In [ ]:
data/ravdess_30_video.npz full cnn architecture
testing_score loss: 1.0944909354050953
testing_score accuracy: 0.7864583333333334